<a href="https://colab.research.google.com/github/sujan123123/CNN/blob/master/twoplustwo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch as th
import torch
from torch import nn
th.cuda.is_available()
import torchvision as tv
from PIL import Image
import numpy as np
import random
import tqdm

In [ ]:
ngpu=1

nz = 1
ngf = 64
ndf = 64
nc = 1
device='cuda'

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu=1):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 4, 4, 1, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 4, 4, 1, 2, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 1, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 1, 2, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 1, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        output = self.main(input.unsqueeze(1))
        output = output[:,0, :28, :28]
        return output


netG = Generator(ngpu).to(device)
netG.apply(weights_init)
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(1, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (13):

In [ ]:
dataset = tv.datasets.MNIST(root='./data', download=True,
                           transform=tv.transforms.Compose([
                               tv.transforms.ToTensor(),
                               tv.transforms.Normalize((0.5,), (0.5,)),
                           ]))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def pimg(img):
  return Image.fromarray((img.detach().cpu().squeeze().numpy()*255).astype('uint8'))


In [ ]:
Image.fromarray(dataset[5][0][0].numpy().astype('uint8'))

In [ ]:
twos = [d for d in dataset if d[1] == 2]

In [ ]:
pimg(twos[0][0])

In [ ]:
num_steps = 9000
batch_size = 64

In [ ]:
plus = th.zeros_like(twos[0][0])
xcoord, ycoord = th.meshgrid(th.range(0, 27), th.range(0, 27))
pidcs = (xcoord >= 6) & (xcoord < 22) & (ycoord >= 12) & (ycoord < 17)
plus[pidcs[None]] = 1
plus[pidcs.T[None]] = 1
plus = plus[0]
pimg(plus)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  


In [ ]:
datasets = [[d for d in dataset if d[1] == c] for c in range(10)]

In [ ]:
optimizerG = th.optim.Adam(netG.parameters(), lr=3e-4, betas=(0.9, 0.999))

In [ ]:
for step in tqdm.trange(num_steps):
  batch = []
  labels = []
  for b in range(batch_size):
    two1, two2 = np.random.choice(len(twos), size=2)
    two1, two2 = twos[two1], twos[two2]
    tpt = th.cat((two1[0][0], plus, two2[0][0]), 1).to(device)
    l1 = two1[1]
    l2 = two2[1]
    label = l1 + l2
    label_ds = datasets[label]
    label = random.choice(label_ds)[0][0].to(device)
    batch.append(tpt)
    labels.append(label)
  batch = th.stack(batch)
  labels = th.stack(labels)
  output = netG(batch)

  netG.zero_grad()
  loss = (output - labels).square().sum(-1).sum(-1).mean()
  loss.backward()
  optimizerG.step()
  if step % 1000 == 0:
    print(loss)

  0%|          | 1/9000 [00:00<1:40:46,  1.49it/s]

tensor(353.8228, device='cuda:0', grad_fn=<MeanBackward0>)


 11%|█         | 1001/9000 [10:24<1:47:08,  1.24it/s]

tensor(166.9863, device='cuda:0', grad_fn=<MeanBackward0>)


 22%|██▏       | 2001/9000 [20:46<1:33:55,  1.24it/s]

...

 78%|███████▊  | 7001/9000 [1:12:36<26:45,  1.24it/s]

tensor(175.5981, device='cuda:0', grad_fn=<MeanBackward0>)


 89%|████████▉ | 8001/9000 [1:22:59<13:23,  1.24it/s]

tensor(166.7821, device='cuda:0', grad_fn=<MeanBackward0>)


100%|██████████| 9000/9000 [1:33:22<00:00,  1.61it/s]


In [ ]:

test_dataset = tv.datasets.MNIST(root='./data', train=False, download=True,
                           transform=tv.transforms.Compose([
                               tv.transforms.ToTensor(),
                               tv.transforms.Normalize((0.5,), (0.5,)),
                           ]))

In [ ]:
test_twos = [d[0][0] for d in test_dataset if d[1] == 2]

In [ ]:
test_two1, test_two2 = test_twos[0], test_twos[1]

In [ ]:
pimg(test_two1)

In [ ]:

pimg(test_two2)

In [ ]:

test_tpt = th.cat((test_two1, plus, test_two2), 1).to(device)
pimg(test_tpt)

In [ ]:
FINAL_OUTPUT = netG(test_tpt.unsqueeze(0))[0]
FINAL_OUTPUT.shape

torch.Size([28, 28])

In [ ]:
pimg(FINAL_OUTPUT)